# Proposition 21: Vehicle License Fee for State Parks

## The way it is now
California has [`park_count` state parks](#:), including state beaches and historic parks. The current [\$`old_budget / 1e6`M](#:) budget is insufficient to maintain these parks, and [`old_closed_park_count` parks](#:) will be shut down at least part-time. Most parks charge [\$`old_admission` per vehicle](#:) for admission.

## What Prop 21 would do
Proposes to charge car owners an extra \$18 on their annual registration bill, to go into the state park fund. Cars that pay the charge would have free park admission.

## Analysis
> Suppose that an [extra  \$`tax`](#:tax) was charged to [`compliance`%](#:compliance) of [`is_tax_per_vehicle`](#:is_tax_per_vehicle). Park admission would be [`new_admission ? '$' + new_admission : "free"`](#:new_admission) for [`applies_to_everyone`](#:applies_to_everyone).

This would collect an extra [`$(delta_budget)`](#:) ([`$(tax_collected)`](#:) from the tax, minus [`$(delta_revenue)`](#:)
[`delta_revenue < 0`](#:if) lost [](#:endif)
[`delta_revenue > 0`](#:if) additional [](#:endif) revenue from admission) for a total state park budget of [`$(budget)`M](#:).

[`budget < maintainance_budget`](#:if)
This is not sufficient to maintain the parks, and [`floor(closed_park_count)` parks](#:) would be shut down at least part-time.
[`budget < repair_budget`](#:elsif)
This is sufficient to maintain the parks in their current state, but not fund a program to bring safety and cleanliness up to acceptable standards. [](#:endif)
[`budget < max_budget`](#:if)
This is sufficient to maintain the parks in their current state, plus fund a program to bring safety and cleanliness up to acceptable standards over the next [`restoration_time` years](#:)
[](#:else)
That is ridiculous. It's not even funny.
[](#:endif)


Park attendance would rise by [`floor(relative_visitor_count * 100)`%](#:), to [`floor(new_visitor_count / 1e6)`M visits each year](#:).

In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
from math import atan
from ipytangle import tangle

In [7]:
itpv_options = ["California taxpayers", "vehicle registrations"]
a2e_options = ["those who paid the charge", "everyone"]


def tax_count(is_tax_per_vehicle, registered_vehicles, taxpayers):
    return registered_vehicles if (is_tax_per_vehicle == itpv_options[1]) else taxpayers

def tax_collected(tax, compliance, tax_count):
    return tax * (compliance / 100) * tax_count

def eligible_new(applies_to_everyone, visitors_instate, is_tax_per_vehicle, vehicle_owners):
    if applies_to_everyone == a2e_options[1]:
        return 1
    return (visitors_instate /  100) * (
        is_tax_per_vehicle == itpv_options[1] or vehicle_owners / 100)

def avg_admission(old_admission, eligible_new, new_admission):
    return old_admission + eligible_new * (new_admission - old_admission)

def new_visitor_count(old_visitor_count, avg_admission, old_admission):
    return old_visitor_count * max(
        0.2,
        1 + 0.5 * atan(1 - avg_admission / old_admission)
    )

def delta_revenue(old_visitor_count, new_visitor_count, old_admission,
                  admission_to_revenue, avg_admission):
    old_rev = old_visitor_count * old_admission * admission_to_revenue
    new_rev = new_visitor_count * avg_admission * admission_to_revenue
    return new_rev - old_rev

def delta_budget(tax_collected, delta_revenue):
    return tax_collected + delta_revenue

def delta_visitor_count(new_visitor_count, old_visitor_count):
    return new_visitor_count - old_visitor_count

def relative_visitor_count(delta_visitor_count, old_visitor_count):
    return abs(delta_visitor_count / old_visitor_count)

def budget(old_budget, delta_budget):
    return old_budget + delta_budget

def closed_park_count(old_closed_park_count, maintainance_budget, budget, old_budget):
    return old_closed_park_count * (maintainance_budget - budget) / maintainance_budget - old_budget 

def restoration_time(budget, repair_budget, max_budget):
    return 10 - 9 * (budget - repair_budget) / (max_budget - repair_budget)

In [8]:
prop21 = tangle(
    # givens
    park_count=278,
    old_admission=12,
    registered_vehicles = 28e6,
    taxpayers=13657632,
    old_visitor_count=75e6,
    old_budget=400e6,
    old_closed_park_count=150,
    admission_to_revenue=0.1,
    visitors_instate=85,
    vehicle_owners=95,
    maintainance_budget=600e6,
    repair_budget=750e6,
    max_budget=1000e6,
    
    # controls
    tax=18,
    compliance=100,
    new_admission=0,
    is_tax_per_vehicle=itpv_options,
    applies_to_everyone=a2e_options,
    # derived values
    tax_count=(0, tax_count),
    tax_collected=(0.0, tax_collected),
    eligible_new=(0.0, eligible_new),
    avg_admission=(0, avg_admission),
    new_visitor_count=(0, new_visitor_count),
    delta_revenue=(0.0, delta_revenue),
    delta_budget=(0.0, delta_budget),
    delta_visitor_count=(0.0, delta_visitor_count),
    relative_visitor_count=(0.0, relative_visitor_count),
    budget=(0.0, budget),
    closed_park_count=(0, closed_park_count),
    restoration_time=(0, restoration_time)
)
prop21